<h1>Lasso model<span class="tocSkip"></span></h1>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#Build-model" data-toc-modified-id="Build-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Build model</a></span></li><li><span><a href="#Predict" data-toc-modified-id="Predict-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Predict</a></span></li></ul></div>

# Introduction 

This notebook will build a lasso model to predict yeast Diamide phneotype based on genotypes. The phenotypes values were binarized, 0 for negative value and 1 for positive value, while the NAs were ignored. 

The parameter lambda is optimized by using cross-validation. 

In [1]:
library('fdrtool')
library('Matrix')
library('foreach')
library('glmnet')
library('caTools')
library('mltools')
library('caret')

set.seed(28217)

Loaded glmnet 2.0-16

Loading required package: lattice
Loading required package: ggplot2


In [2]:
normalize <- function(x) {
    return ((x - min(x)) / (max(x) - min(x)))
  }

# Load data

In [3]:
# load data
genotype_file <- 'data/genotype_full.txt' 
genotype <- read.table(
  file = genotype_file,
  sep = '\t',
  header = TRUE,
  check.names = FALSE,
  row.names = 1
)
sprintf('genotype size: (%d, %d)', nrow(genotype), ncol(genotype))

[1] "genotype size: (4390, 28220)"

In [4]:
head(genotype)

ERROR while rich displaying an object: Error in sprintf(wrap, caption, header, body): 'fmt' length exceeds maximal format length 8192

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. repr::mime2repr[[mime]]

,33070_chrI_33070_A_T,33147_chrI_33147_G_T,33152_chrI_33152_T_C,33200_chrI_33200_C_T,33293_chrI_33293_A_T,33328_chrI_33328_C_A,33348_chrI_33348_G_C,33403_chrI_33403_C_T,33502_chrI_33502_A_G,33548_chrI_33548_A_C,⋯,12048853_chrXVI_925593_G_C,12049199_chrXVI_925939_T_C,12049441_chrXVI_926181_C_T,12050613_chrXVI_927353_T_G,12051167_chrXVI_927907_A_C,12051240_chrXVI_927980_A_G,12051367_chrXVI_928107_C_T,12052782_chrXVI_929522_C_T,12052988_chrXVI_929728_A_G,12053130_chrXVI_929870_C_T
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
01_01,1,1,1,1,1,1,1,1,1,1,⋯,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
01_02,1,1,1,1,1,1,1,1,1,1,⋯,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
01_03,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,⋯,1,1,1,1,1,1,1,1,1,1
01_04,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,1
01_06,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,⋯,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
01_07,1,1,1,1,1,1,1,1,1,1,⋯,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [5]:
phenotype_file <- 'data/phenotype.csv'
multi_pheno <- read.table(
  file = phenotype_file,
  sep = ',',
  header = TRUE,
  row.names = 1
)
sprintf('multi_pheno size: (%d, %d)', nrow(multi_pheno), ncol(multi_pheno))

[1] "multi_pheno size: (4390, 20)"

In [6]:
head(multi_pheno)

,X1_CobaltChloride_1,X1_CopperSulfate_1,X1_Diamide_1,X1_E6.Berbamine_1,X1_Ethanol_1,X1_Formamide_1,X1_Hydroxyurea_1,X1_IndolaceticAcid_1,X1_Lactate_1,X1_Lactose_1,X1_MagnesiumChloride_1,X1_ManganeseSulfate_1,X1_Menadione_1,X1_Neomycin_1,X1_Raffinose_1,X1_Trehalose_1,X1_Xylose_1,X1_YNB_1,X1_YPD_1,X1_Zeocin_1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
01_01,-2.253831,-1.5881462,0.1949297,-1.05593834,-0.2503701,0.4982271,-0.2132440,-0.1818653,NA,-0.84758635,-0.352480856,1.2121624,0.33522416,-0.6652690,-0.37046982,-0.6748265,-0.8169721,17.71107,25.87121,0.76390775
01_02,-1.887746,0.5428723,0.4515405,0.01159349,0.1037188,0.8286596,0.6391118,0.6608202,NA,-0.62045976,0.394129096,-1.9428571,1.39795249,-0.3139357,1.00710224,0.4933509,-1.4124154,18.28669,26.21880,1.27211199
01_03,1.047185,0.4530668,0.7218348,1.64530117,0.4276157,-0.3261767,-0.1417717,-0.6118751,-0.7977368,-0.21919298,-0.108410797,0.7501782,-0.91339523,0.4199074,-0.07218847,-0.3467727,0.1695682,15.49954,24.49684,0.07232283
01_04,2.417437,0.7474267,0.4545174,1.85680865,-0.1357309,0.5565142,0.1972328,0.3711080,NA,0.66606788,0.021487376,-0.9172175,-0.23938559,0.7443189,0.03371876,1.7741855,0.6684001,17.30108,25.82781,0.67644666
01_06,-1.041743,0.1803843,0.4644736,-0.96622480,-0.3380302,-0.7282211,0.5434985,-1.8339306,-0.1702993,0.08603016,0.108120016,-1.2516301,-0.03877182,-0.6707914,-0.23361657,-0.1999031,-0.2834708,15.30869,25.51335,0.99602726
01_07,1.734380,0.4409412,0.3804743,-0.04976184,0.2623285,-1.0056239,0.5271231,-0.6569152,-0.3989942,-0.67089428,-0.003433968,-1.1416733,-0.92093009,0.8349070,-0.82728217,-0.4337953,0.9380309,15.43720,24.15441,-0.81202630


In [7]:
X <- genotype
Y <- multi_pheno[, 3]

# take a small part to test code
# X <- genotype[1:1000, 1:5000]
# Y <- multi_pheno[1:1000, 1]

# Preprocessing

In [8]:
# remove the gene loci with NA traits
X <- X[!is.na(Y), ]
Y <- Y[!is.na(Y)]

dim(X)
length(Y)

[1]  4309 28220

[1] 4309

In [9]:
# binarize
Y[Y > 0] = 1
Y[Y < 0] = 0

table(Y)

Y
   0    1 
1881 2428 

In [10]:
# generate balance dataset
majority <- 1 
minority <- 0

X_majority <- X[Y == majority, ]
X_minority <- X[Y == minority, ]

index_downsample <- sample(seq_len(length(Y[Y == majority])), length(Y[Y == minority]))
X_downsampled <- X_majority[index_downsample,]

X_balanced <- rbind(X_minority, X_downsampled)
dim(X_balanced)

Y_minority <- as.numeric(rep(minority, length(Y[Y == minority])))
Y_majority <- as.numeric(rep(majority, length(Y[Y == minority])))
Y_balanced <- c(Y_minority, Y_majority)
length(Y_balanced)

[1]  3762 28220

[1] 3762

In [11]:
## split train and test
smp_size <- floor(0.9 * nrow(X_balanced))
train_ind <- sample(seq_len(nrow(X_balanced)), size = smp_size)

X_train <- X_balanced[train_ind, ]
X_test <- X_balanced[-train_ind, ]
Y_train <- Y_balanced[train_ind]
Y_test <- Y_balanced[-train_ind]

dim(X_train) 
dim(X_test)

[1]  3385 28220

[1]   377 28220

# Build model

In [12]:
# cross validation to optimize lasso
cv_fit <- cv.glmnet(as.matrix(X_train), as.matrix(Y_train), nfolds=5, family=c("binomial"))

cv_fit$lambda.min

[1] 0.006439069

In [13]:
Lasso_fit <- glmnet(as.matrix(X_train), as.matrix(Y_train), alpha = 1, family = c("binomial"), 
    lambda = cv_fit$lambda.min, intercept = TRUE)

Lasso_fit


Call:  glmnet(x = as.matrix(X_train), y = as.matrix(Y_train), family = c("binomial"),      alpha = 1, lambda = cv_fit$lambda.min, intercept = TRUE) 

      Df   %Dev   Lambda
[1,] 271 0.4937 0.006439

In [14]:
# save the model to disk
saveRDS(Lasso_fit, "models/Lasso_model.rds")

# Loaded_model <- readRDS("./Lasso_model.rds")
# print(Loaded_model)

# Predict

In [15]:
# predict
Y_predicted <- predict(Lasso_fit, type = "class", newx = as.matrix(X_test))

In [16]:
confusionMatrix(table(Y_test, Y_predicted))

Confusion Matrix and Statistics

      Y_predicted
Y_test   0   1
     0 165  32
     1  42 138
                                        
               Accuracy : 0.8037        
                 95% CI : (0.76, 0.8426)
    No Information Rate : 0.5491        
    P-Value [Acc > NIR] : <2e-16        
                                        
                  Kappa : 0.6057        
                                        
 Mcnemar's Test P-Value : 0.2955        
                                        
            Sensitivity : 0.7971        
            Specificity : 0.8118        
         Pos Pred Value : 0.8376        
         Neg Pred Value : 0.7667        
             Prevalence : 0.5491        
         Detection Rate : 0.4377        
   Detection Prevalence : 0.5225        
      Balanced Accuracy : 0.8044        
                                        
       'Positive' Class : 0             
                                        